# Lectura de archivos

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1RUtolRcQlR3RGULttM4ZoQaK_Ouow4gc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')

In [4]:
train_labels = pd.read_csv('train_labels.csv', encoding='latin-1', dtype={'building_id': 'int64', 'damage_grade': 'int64'}, index_col='building_id')

In [5]:
##1RUtolRcQlR3RGULttM4ZoQaK_Ouow4gc
id='1br3fMwXX_J0XmiXvOm_wfKWvHSj45T3y'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values.csv')

In [6]:
train_values = pd.read_csv('train_values.csv', encoding='latin-1', dtype = {'building_id': 'int64', 'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',\
                                                                           'count_floors_pre_eq': 'int64', 'age': 'int64', 'area_percentage': 'int64', \
                                                                           'height_percentage': 'int64', 'land_surface_condition': 'category',\
                                                                           'foundation_type': 'category', 'roof_type': 'category', 'ground_floor_type': 'category',\
                                                                           'other_floor_type': 'category', 'position': 'category', 'plan_configuration': 'category',\
                                                                           'has_superstructure_adobe_mud': 'boolean', 'has_superstructure_mud_mortar_stone': 'boolean', \
                                                                           'has_superstructure_stone_flag': 'boolean', 'has_superstructure_cement_mortar_stone': 'boolean',\
                                                                           'has_superstructure_mud_mortar_brick': 'boolean', 'has_superstructure_cement_mortar_brick': 'boolean',\
                                                                           'has_superstructure_timber': 'boolean', 'has_superstructure_bamboo': 'boolean',\
                                                                           'has_superstructure_rc_non_engineered': 'boolean', 'has_superstructure_rc_engineered': 'boolean',\
                                                                           'has_superstructure_other': 'boolean', 'legal_ownership_status': 'category', 'count_families': 'int64', \
                                                                           'has_secondary_use': 'boolean', 'has_secondary_use_agriculture': 'boolean', 'has_secondary_use_hotel': 'boolean', \
                                                                           'has_secondary_use_rental': 'boolean', 'has_secondary_use_institution': 'boolean', 'has_secondary_use_school': 'boolean',\
                                                                           'has_secondary_use_industry': 'boolean', 'has_secondary_use_health_post': 'boolean', \
                                                                           'has_secondary_use_gov_office': 'boolean', 'has_secondary_use_use_police': 'boolean', 'has_secondary_use_other': 'boolean'}, index_col='building_id')

In [7]:
id = '1kt2VFhgpfRS72wtBOBy1KDat9LanfMZU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values.csv')

In [8]:
test_values = pd.read_csv('test_values.csv', encoding='latin-1', dtype = {'building_id': 'int64', 'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',\
                                                                           'count_floors_pre_eq': 'int64', 'age': 'int64', 'area_percentage': 'int64', \
                                                                           'height_percentage': 'int64', 'land_surface_condition': 'category',\
                                                                           'foundation_type': 'category', 'roof_type': 'category', 'ground_floor_type': 'category',\
                                                                           'other_floor_type': 'category', 'position': 'category', 'plan_configuration': 'category',\
                                                                           'has_superstructure_adobe_mud': 'boolean', 'has_superstructure_mud_mortar_stone': 'boolean', \
                                                                           'has_superstructure_stone_flag': 'boolean', 'has_superstructure_cement_mortar_stone': 'boolean',\
                                                                           'has_superstructure_mud_mortar_brick': 'boolean', 'has_superstructure_cement_mortar_brick': 'boolean',\
                                                                           'has_superstructure_timber': 'boolean', 'has_superstructure_bamboo': 'boolean',\
                                                                           'has_superstructure_rc_non_engineered': 'boolean', 'has_superstructure_rc_engineered': 'boolean',\
                                                                           'has_superstructure_other': 'boolean', 'legal_ownership_status': 'category', 'count_families': 'int64', \
                                                                           'has_secondary_use': 'boolean', 'has_secondary_use_agriculture': 'boolean', 'has_secondary_use_hotel': 'boolean', \
                                                                           'has_secondary_use_rental': 'boolean', 'has_secondary_use_institution': 'boolean', 'has_secondary_use_school': 'boolean',\
                                                                           'has_secondary_use_industry': 'boolean', 'has_secondary_use_health_post': 'boolean', \
                                                                           'has_secondary_use_gov_office': 'boolean', 'has_secondary_use_use_police': 'boolean', 'has_secondary_use_other': 'boolean'})

In [85]:
train_set = train_values.merge(train_labels, on='building_id')
test_set = test_values

# Feature Engineering

## Impacto en su zona geográfica:

In [63]:
geo_levels = train_set[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'damage_grade']].reset_index()
geo_levels.head(10)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,damage_grade
0,802906,6,487,12198,3
1,28830,8,900,2812,2
2,94947,21,363,8973,3
3,590882,22,418,10694,2
4,201944,11,131,1488,3
5,333020,8,558,6089,2
6,728451,9,475,12066,3
7,475515,20,323,12236,1
8,441126,0,757,7219,2
9,989500,26,886,994,1


### Promedio del daño en cada zona:

In [64]:
#media de daño por cada zona
damage_avg_1 = geo_levels.groupby('geo_level_1_id')['damage_grade'].mean()
damage_avg_2 = geo_levels.groupby('geo_level_2_id')['damage_grade'].mean()
damage_avg_3 = geo_levels.groupby('geo_level_3_id')['damage_grade'].mean()

In [65]:
damage_avg_1.rename('damage_avg_geo_level_1_id', inplace=True)
damage_avg_2.rename('damage_avg_geo_level_2_id', inplace=True)
damage_avg_3.rename('damage_avg_geo_level_3_id', inplace=True)

geo_level_3_id
0        3.000000
1        2.500000
3        1.888889
5        1.142857
6        2.523810
           ...   
12562    2.000000
12563    2.041667
12564    2.000000
12565    1.285714
12567    2.000000
Name: damage_avg_geo_level_3_id, Length: 11595, dtype: float64

In [66]:
geo_levels = geo_levels\
        .merge(damage_avg_1, on='geo_level_1_id')\
        .merge(damage_avg_2, on='geo_level_2_id')\
        .merge(damage_avg_3, on='geo_level_3_id')

### Qué daño es mayoritario en cada zona:

In [67]:
#valor más común de daño por zona
damage_mode_1 = geo_levels.groupby('geo_level_1_id')['damage_grade'].agg(lambda x:x.mode())
damage_mode_2 = geo_levels.groupby('geo_level_2_id').agg({'damage_grade': lambda x: list(x.mode())})['damage_grade'].map(lambda x: x[0])
damage_mode_3 = geo_levels.groupby('geo_level_3_id').agg({'damage_grade': lambda x: list(x.mode())})['damage_grade'].map(lambda x: x[0])

In [68]:
damage_mode_1.rename('damage_mode_geo_level_1_id', inplace=True)
damage_mode_2.rename('damage_mode_geo_level_2_id', inplace=True)
damage_mode_3.rename('damage_mode_geo_level_3_id', inplace=True)

geo_level_3_id
0        3
1        3
3        2
5        1
6        3
        ..
12562    2
12563    2
12564    2
12565    1
12567    2
Name: damage_mode_geo_level_3_id, Length: 11595, dtype: int64

In [69]:
geo_levels = geo_levels\
        .merge(damage_mode_1, on='geo_level_1_id')\
        .merge(damage_mode_2, on='geo_level_2_id')\
        .merge(damage_mode_3, on='geo_level_3_id')

### Promedio de cuánto hay de cada daño por cada zona:

In [70]:
#porcentaje de edificios con daño 1 por zona
damage_percentage_1 = geo_levels\
        .groupby('geo_level_1_id')\
        .agg({'damage_grade': [lambda x: (x==1).mean(), lambda x: (x==2).mean(), lambda x: (x==3).mean()]})

damage_percentage_2 = geo_levels\
        .groupby('geo_level_2_id')\
        .agg({'damage_grade': [lambda x: (x==1).mean(), lambda x: (x==2).mean(), lambda x: (x==3).mean()]})

damage_percentage_3 = geo_levels\
        .groupby('geo_level_3_id')\
        .agg({'damage_grade': [lambda x: (x==1).mean(), lambda x: (x==2).mean(), lambda x: (x==3).mean()]})

In [71]:
#para no compilar varias veces lo de arriba porque tarda
dm_prct1 = damage_percentage_1
dm_prct2 = damage_percentage_2
dm_prct3 = damage_percentage_3

In [72]:
dmg_percentage = [dm_prct1, dm_prct2, dm_prct3]

for i in [0,1,2]:
  damage = dmg_percentage[i]
  damage.columns = damage.columns.get_level_values(0) + damage.columns.get_level_values(1)
  damage\
  .rename(columns={'damage_grade<lambda_0>': 'percentage_damage_1_level_'+str(i+1)+'_id',\
                   'damage_grade<lambda_1>': 'percentage_damage_2_level_'+str(i+1)+'_id', \
                   'damage_grade<lambda_2>': 'percentage_damage_3_level_'+str(i+1)+'_id'}, inplace=True)
  damage.reset_index(inplace=True)



In [75]:
dm_prct3.head(1)

,geo_level_3_id,percentage_damage_1_level_3_id,percentage_damage_2_level_3_id,percentage_damage_3_level_3_id
0,0,0.0,0.0,1.0


In [76]:
geo_levels= geo_levels\
        .merge(dm_prct1, on='geo_level_1_id')\
        .merge(dm_prct2, on='geo_level_2_id')\
        .merge(dm_prct3, on='geo_level_3_id')

In [77]:
geo_levels.head(5)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,damage_grade,damage_avg_geo_level_1_id,damage_avg_geo_level_2_id,damage_avg_geo_level_3_id,damage_mode_geo_level_1_id,damage_mode_geo_level_2_id,damage_mode_geo_level_3_id,percentage_damage_1_level_1_id,percentage_damage_2_level_1_id,percentage_damage_3_level_1_id,percentage_damage_1_level_2_id,percentage_damage_2_level_2_id,percentage_damage_3_level_2_id,percentage_damage_1_level_3_id,percentage_damage_2_level_3_id,percentage_damage_3_level_3_id
0,802906,6,487,12198,3,2.161724,2.740741,2.837838,2,3,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.0,0.162162,0.837838
1,959468,6,487,12198,3,2.161724,2.740741,2.837838,2,3,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.0,0.162162,0.837838
2,215003,6,487,12198,3,2.161724,2.740741,2.837838,2,3,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.0,0.162162,0.837838
3,302373,6,487,12198,3,2.161724,2.740741,2.837838,2,3,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.0,0.162162,0.837838
4,820534,6,487,12198,3,2.161724,2.740741,2.837838,2,3,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.0,0.162162,0.837838


### Agrego los nuevos Features

In [96]:
new_features = ['building_id', 'damage_avg_geo_level_1_id', 'damage_avg_geo_level_2_id', 'damage_avg_geo_level_3_id',\
                'damage_mode_geo_level_1_id', 'damage_mode_geo_level_2_id', 'damage_mode_geo_level_3_id',\
                'percentage_damage_1_level_1_id', 'percentage_damage_2_level_1_id', 'percentage_damage_3_level_1_id',
                'percentage_damage_1_level_2_id', 'percentage_damage_2_level_2_id', 'percentage_damage_3_level_2_id',
                'percentage_damage_1_level_3_id', 'percentage_damage_2_level_3_id', 'percentage_damage_3_level_3_id'
                ]
geo_levels.reset_index(inplace=True)
train_set.reset_index(inplace=True)

train_set = train_set.merge(geo_levels[new_features], on='building_id')
train_set.head(5)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,damage_avg_geo_level_1_id,damage_avg_geo_level_2_id,damage_avg_geo_level_3_id,damage_mode_geo_level_1_id,damage_mode_geo_level_2_id,damage_mode_geo_level_3_id,percentage_damage_1_level_1_id,percentage_damage_2_level_1_id,percentage_damage_3_level_1_id,percentage_damage_1_level_2_id,percentage_damage_2_level_2_id,percentage_damage_3_level_2_id,percentage_damage_1_level_3_id,percentage_damage_2_level_3_id,percentage_damage_3_level_3_id
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,True,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3,2.161724,2.740741,2.837838,2,3,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.000000,0.162162,0.837838
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2,2.485273,2.487437,2.062500,3,3,2,0.034277,0.446174,0.519549,0.010050,0.492462,0.497487,0.062500,0.812500,0.125000
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3,2.563369,2.518750,2.580882,3,3,3,0.021627,0.393378,0.584996,0.082386,0.316477,0.601136,0.029412,0.360294,0.610294
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,False,True,False,False,False,False,True,True,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2,2.000960,2.107317,2.096774,2,2,2,0.129718,0.739603,0.130678,0.019512,0.853659,0.126829,0.032258,0.838710,0.129032
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,True,False,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3,2.337713,2.348748,2.368852,2,2,2,0.046959,0.568370,0.384672,0.029865,0.591522,0.378613,0.008197,0.614754,0.377049


## Estado al momento de la construcción

In [ ]:
#land_surface_condition, foundation_type, roof_type, ground_floor_type, other_floor_type, position, plan_configuration